# Share of organic area in total utilized agricultural area (UAA)

Reproducing data from Eurostat's article [Organic farming statistics](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Organic_farming_statistics).

Data are taken from two datasets:
* Organic crop area by agricultural production methods and crops (from 2012 onwards) (ORG_CROPAR)
* Utilised agricultural area by categories (TAG00025).

It is possible to get and customize the API links from the Data Browser in "Download" and "Options and other formats", e.g. for the [Organic crop area by agricultural production methods and crops ](https://ec.europa.eu/eurostat/databrowser/view/org_cropar/) dataset.

![data-browser-screenshot.png](images/data-browser-screenshot.png)


In [1]:
import pandas as pd

In [2]:
# ORG_CROPAR 
# Organic crop area by agricultural production methods and crops (from 2012 onwards)
org_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/ORG_CROPAR$DEFAULTVIEW/A.HA.UAAXK0000.TOTAL.EU+EU27_2020+EU28+BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE+IS+NO+CH+UK+MK+RS+TR/?format=SDMX-CSV&startPeriod=2012&endPeriod=2020'
# TAG00025
# Utilised agricultural area by categories
uaa_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/TAG00025/A.UAA.MA.BE+BG+CZ+DK+DE+EE+IE+EL+ES+FR+HR+IT+CY+LV+LT+LU+HU+MT+NL+AT+PL+PT+RO+SI+SK+FI+SE+IS+LI+NO+CH+UK+ME+MK+AL+RS+TR+BA+XK/?format=SDMX-CSV&startPeriod=2009&endPeriod=2020'

In [3]:
df = pd.read_csv(org_url)
df.head()

,DATAFLOW,LAST UPDATE,freq,unit,crops,agprdmet,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:ORG_CROPAR$DEFAULTVIEW(1.0),04/08/21 23:00:00,A,HA,UAAXK0000,TOTAL,AT,2012,533230,NaN
1,ESTAT:ORG_CROPAR$DEFAULTVIEW(1.0),04/08/21 23:00:00,A,HA,UAAXK0000,TOTAL,AT,2013,526689,NaN
2,ESTAT:ORG_CROPAR$DEFAULTVIEW(1.0),04/08/21 23:00:00,A,HA,UAAXK0000,TOTAL,AT,2014,525521,NaN
3,ESTAT:ORG_CROPAR$DEFAULTVIEW(1.0),04/08/21 23:00:00,A,HA,UAAXK0000,TOTAL,AT,2015,552141,NaN
4,ESTAT:ORG_CROPAR$DEFAULTVIEW(1.0),04/08/21 23:00:00,A,HA,UAAXK0000,TOTAL,AT,2016,571423,NaN


In [4]:
org_area = df.loc[df['TIME_PERIOD'] == 2019, ['geo', 'OBS_VALUE']].copy()

In [5]:
org_area.columns = ['geo', 'org_area']
# Correct Eurostat's encoding of UK and Greece
org_area.loc[org_area['geo'] == 'EL', ['geo']] = 'GR'
org_area.loc[org_area['geo'] == 'UK', ['geo']] = 'GB'
org_area.set_index('geo', inplace=True)
org_area.head(10)

,org_area
geo,
AT,671703
BE,93119
BG,117779
CH,169030
CY,6240
CZ,535185
DE,1290839
DK,291247
EE,220737


In [6]:
df2 = pd.read_csv(uaa_url)
df2.head()

,DATAFLOW,LAST UPDATE,freq,crops,strucpro,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:TAG00025(1.0),17/09/21 23:00:00,A,UAA,MA,AL,2009,1200.9,NaN
1,ESTAT:TAG00025(1.0),17/09/21 23:00:00,A,UAA,MA,AL,2010,1200.9,NaN
2,ESTAT:TAG00025(1.0),17/09/21 23:00:00,A,UAA,MA,AL,2011,1201.3,NaN
3,ESTAT:TAG00025(1.0),17/09/21 23:00:00,A,UAA,MA,AL,2012,1195.3,NaN
4,ESTAT:TAG00025(1.0),17/09/21 23:00:00,A,UAA,MA,AL,2014,1174.3,NaN


In [7]:
uaa = df2.loc[df2['TIME_PERIOD'] == 2019, ['geo', 'OBS_VALUE']].copy()

In [8]:
uaa.columns = ['geo', 'uaa']
uaa.loc[uaa['geo'] == 'EL', ['geo']] = 'GR'
uaa.loc[uaa['geo'] == 'UK', ['geo']] = 'GB'
uaa.set_index('geo', inplace=True)
uaa['uaa'] = uaa['uaa'] * 1000
uaa['uaa'] = uaa.astype({'uaa': 'int'})
uaa.head()

,uaa
geo,
AL,1174100
AT,2652220
BA,1806270
BE,1358700
BG,5037470


In [9]:
org_share = org_area.merge(uaa, on='geo')
org_share.head()

,org_area,uaa
geo,,
AT,671703,2652220
BE,93119,1358700
BG,117779,5037470
CH,169030,1512100
CY,6240,125350


In [10]:
org_share['organic_share'] = round((org_share['org_area'] / org_share['uaa']) * 100, 2)
org_share.head(6)

,org_area,uaa,organic_share
geo,,,
AT,671703,2652220,25.33
BE,93119,1358700,6.85
BG,117779,5037470,2.34
CH,169030,1512100,11.18
CY,6240,125350,4.98
CZ,535185,3523660,15.19


In [11]:
org_share.sort_index().to_csv('organic_share_all.csv', index_label='geo')

In [12]:
org_share.sort_index().loc[:, ['organic_share']].to_csv('organic_share.csv', index_label='geo')